# 📊 Cardiac Shape Statistical Analysis

This notebook analyzes shape coefficients from a SSM, linking them to demographic and physiological variables.

---

### Covered Steps:
1. Load shape data and demographic metadata
2. Explore distributions and check multivariate assumptions
3. Compare male vs female shapes using Hotelling's T² test
4. Test variance homogeneity (Levene's test)
5. Visualize shape mode distributions (violin plots)
6. Perform multivariate analysis of variance (MANCOVA)
7. Correct the shape coefficients for significant covariates
8. Perform logistic regression analysis

## 📦 Imports and Setup

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import levene, ttest_ind
from hotelling.stats import hotelling_t2
import pyvista as pv
import statsmodels.api as sm
from matplotlib import cm  
import statsmodels.api as sm
from sklearn.metrics import roc_curve, roc_auc_score

color_pallete = ['#440154', '#3a528b', '#21918c', '#5dc962', '#bade28']

## 📂 Load population and shape data

In [ ]:
# === File paths ===
population_file = "path/to/population_metadata.xlsx"
shape_score_file = "path/to/model/shape_coefficients.csv"
mesh_list_file = "path/to/mesh_cohort.csv"

# === Load population metadata and mesh filenames ===
full_population_info = pd.read_excel(population_file)
mesh_list = pd.read_csv(mesh_list_file)

# === Load and normalize shape coefficients ===
shape_scores = pd.read_csv(shape_score_file, header=None)
shape_scores = (shape_scores - shape_scores.mean()) / shape_scores.std()
shape_scores.columns = [f"Mode {i}" for i in range(1, shape_scores.shape[1] + 1)]

# === Merge shape coefficients with Subject_ID ===
shape_scores.insert(0, "Subject_ID", mesh_list["Subject_ID"])

# === Final combined dataframe ===
full_df = pd.merge(shape_scores, full_population_info, on="Subject_ID")

### ⚧️ Split by Sex

In [ ]:
male_df = full_df[full_df['Sex'] == 'Male']
female_df = full_df[full_df['Sex'] == 'Female']

### 🎻 Violin Plot of Top 8 Modes

In [ ]:
# Melt top 8 modes for violin plot
melted_df = full_df.melt(id_vars=['Sex'], value_vars=[f'Mode {i}' for i in range(1, 21)],
                         var_name='Mode', value_name='Z-score')
modes_1_to_8 = melted_df[melted_df['Mode'].isin([f'Mode {i}' for i in range(1, 9)])]

# Plot
sns.set(style='whitegrid')
plt.figure(figsize=(20, 8))
ax = sns.violinplot(x='Mode', y='Z-score', data=modes_1_to_8, hue='Sex', split=True,
                    palette={'Male': '#21918c', 'Female': '#440154'}, linewidth=1.5, inner=None, cut=0.8)

# Add significance stars
y_max = modes_1_to_8['Z-score'].max()
for i, mode in enumerate([f'Mode {i}' for i in range(1, 9)]):
    group = modes_1_to_8[modes_1_to_8['Mode'] == mode]
    t_stat, p_val = ttest_ind(group[group['Sex'] == 'Male']['Z-score'],
                              group[group['Sex'] == 'Female']['Z-score'])
    stars = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else ''
    if stars:
        ax.text(i, y_max + 0.25, stars, ha='center', va='bottom', fontsize=16)

plt.xlabel('')
plt.ylabel('Normalized shape coefficient', fontsize=18)
plt.xticks(rotation=45, fontsize=14)
plt.yticks(fontsize=14)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='upper right', fontsize=14, frameon=True)
sns.despine(top=True, right=True)
plt.tight_layout()
# plt.savefig('mode_violin_plot.png', dpi=300)
plt.show()

### 🧪 Levene's Test (Homogeneity of Variance)

In [ ]:
dependent_vars = [f'Mode {i}' for i in range(1, 21)]
levene_results = []

for var in dependent_vars:
    stat, p = levene(male_df[var].dropna(), female_df[var].dropna(), center='median')
    levene_results.append((var, stat, p))

levene_df = pd.DataFrame(levene_results, columns=['Variable', 'Levene Statistic', 'P-value'])
levene_df['Variance Homogeneity Violated'] = levene_df['P-value'] < 0.01
levene_df

### 📊 Hotelling’s T² Test Between Sexes

In [ ]:
X = male_df[[f'Mode {i}' for i in range(1, 21)]]
Y = female_df[[f'Mode {i}' for i in range(1, 21)]]

t_squared, f_stat, p_value, pooled_cov = hotelling_t2(X, Y)

print(f"Hotelling's T² statistic: {t_squared:.2f}")
print(f"F-statistic: {f_stat:.2f}")
print(f"P-value: {p_value:.4f}")

### 📈 Multivariate Analysis of Covariance (MANCOVA)

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.multivariate.manova import MANOVA

In [ ]:
from scipy.stats import kstest, norm

# Initialize a list to store the results
non_normal_results = []

for mode in dependent_vars:
    stat, p = kstest(full_df[mode].dropna(), norm.cdf)
    if p < 0.05:
        non_normal_results.append({
            'Variable': mode,
            'Statistic': stat,
            'p-value': p
        })

# Convert results to a DataFrame for better readability

non_normality_results = pd.DataFrame(non_normal_results)

# Display only the variables that don't meet the normality assumption
if not non_normality_results.empty:
    print(non_normality_results)
    print("\nSome dependent variables do not meet the normality assumption.")
    print("Consider using non-parametric tests or transformations.")
else:
    print("All dependent variables meet the normality assumption.")

Performing MANCOVA of the shape coefficients against sex, age, systolic blood pressure (BP) and Body Mass Index (BMI): 

In [ ]:
target_body_size = 'BMI'

# Define the dependent variables
dependent_vars = [f'Mode_{mode}' for mode in range(1, 21)]

# Create formula string including all independent variables
formula = ' + '.join(dependent_vars) + ' ~ Sex_binary + '+ target_body_size +' + Age + Systolic_BP_mean_reading'  

# Perform MANOVA using from_formula
manova = MANOVA.from_formula(formula, data=full_df)

# Original MANOVA test results
original_results = manova.mv_test()

print(original_results)

# Extract p-values for the original data
original_p_values = {var: original_results.results[var]['stat']['Pr > F'].values[0] for var in original_results.results}

# List of independent variables in the model, with 'Sex_binary' renamed to 'Sex'
independent_vars = ['Sex_binary',target_body_size,'Age','Systolic_BP_mean_reading']

# Extract Pillai's trace values for each independent variable
pillai_values = [original_results.results[var]['stat']['Value'].loc['Pillai\'s trace'] for var in independent_vars]

# Create a DataFrame for easier plotting
pillai_df = pd.DataFrame({
    'Variable': independent_vars,
    'Pillai\'s Trace': pillai_values
})

# Use a clean style without grid and background
sns.set_style("white")

# Plot the Pillai's trace values using the Viridis color palette
plt.figure(figsize=(5, 3))
ax = sns.barplot(x=pillai_df['Pillai\'s Trace'], y=pillai_df['Variable'], color=color_pallete[1])

# Add annotations
for p in ax.patches:
    ax.annotate(f'{p.get_width():.2f}', (p.get_width() + 0.01, p.get_y() + p.get_height() / 2),
                ha='left', va='center', fontsize=14)
    ax.axvline(x=0.25, linestyle='--', color='gray', linewidth=1)
    ax.set_xlim(0, 0.6)

# Customize the plot
plt.xlabel('Pillai\'s Trace Value', fontsize=16)
plt.ylabel('')
plt.title('Model 1', fontsize=18)
plt.gca().invert_yaxis()  # To have the first variable at the top
plt.gca().xaxis.set_visible(False)  # Show the x-axis

# Adjust label names in the plot
label_dict = {'Sex_binary': 'Sex', 'Systolic_BP_mean_reading': 'Systolic BP', 'Age': 'Age',  target_body_size: target_body_size}  # Update to any variable names used
ax.set_yticklabels([label_dict.get(label, label) for label in pillai_df['Variable']])

# Remove all spines except the bottom (x-axis)
sns.despine(left=True, right=True, top=True, bottom=True)

# Display the plot
plt.show()

Performing MANCOVA of the shape coefficients against sex, age, systolic blood pressure (BP) and Body Surface Area (BSA): 

In [ ]:
target_body_size = 'BSA'

# Define the dependent variables
dependent_vars = [f'Mode_{mode}' for mode in range(1, 21)]

# Create formula string including all independent variables
formula = ' + '.join(dependent_vars) + ' ~ Sex_binary + '+ target_body_size +' + Age + Systolic_BP_mean_reading'  

# Perform MANOVA using from_formula
manova = MANOVA.from_formula(formula, data=full_df)

# Original MANOVA test results
original_results = manova.mv_test()

print(original_results)

# Extract p-values for the original data
original_p_values = {var: original_results.results[var]['stat']['Pr > F'].values[0] for var in original_results.results}

# List of independent variables in the model, with 'Sex_binary' renamed to 'Sex'
independent_vars = ['Sex_binary',target_body_size,'Age','Systolic_BP_mean_reading']

# Extract Pillai's trace values for each independent variable
pillai_values = [original_results.results[var]['stat']['Value'].loc['Pillai\'s trace'] for var in independent_vars]

# Create a DataFrame for easier plotting
pillai_df = pd.DataFrame({
    'Variable': independent_vars,
    'Pillai\'s Trace': pillai_values
})

# Use a clean style without grid and background
sns.set_style("white")

# Plot the Pillai's trace values using the Viridis color palette
plt.figure(figsize=(5, 3))
ax = sns.barplot(x=pillai_df['Pillai\'s Trace'], y=pillai_df['Variable'], color=color_pallete[2])

# Add annotations
for p in ax.patches:
    ax.annotate(f'{p.get_width():.2f}', (p.get_width() + 0.01, p.get_y() + p.get_height() / 2),
                ha='left', va='center', fontsize=14)
    ax.axvline(x=0.25, linestyle='--', color='gray', linewidth=1)
    ax.set_xlim(0, 0.6)

# Customize the plot
plt.xlabel('Pillai\'s Trace Value', fontsize=16)
plt.ylabel('')
plt.title('Model 2', fontsize=18)
plt.gca().invert_yaxis()  # To have the first variable at the top
plt.gca().xaxis.set_visible(False)  # Show the x-axis

# Adjust label names in the plot
label_dict = {'Sex_binary': 'Sex', 'Systolic_BP_mean_reading': 'Systolic BP', 'Age': 'Age',  target_body_size: target_body_size}  # Update to any variable names used
ax.set_yticklabels([label_dict.get(label, label) for label in pillai_df['Variable']])

# Remove all spines except the bottom (x-axis)
sns.despine(left=True, right=True, top=True, bottom=True)

# Display the plot
plt.show()

Performing MANCOVA of the shape coefficients against sex, age, systolic blood pressure (BP) and Height: 

In [ ]:
target_body_size = 'Height'

# Define the dependent variables
dependent_vars = [f'Mode_{mode}' for mode in range(1, 21)]

# Create formula string including all independent variables
formula = ' + '.join(dependent_vars) + ' ~ Sex_binary + '+ target_body_size +' + Age + Systolic_BP_mean_reading'  

# Perform MANOVA using from_formula
manova = MANOVA.from_formula(formula, data=full_df)

# Original MANOVA test results
original_results = manova.mv_test()

print(original_results)

# Extract p-values for the original data
original_p_values = {var: original_results.results[var]['stat']['Pr > F'].values[0] for var in original_results.results}

# List of independent variables in the model, with 'Sex_binary' renamed to 'Sex'
independent_vars = ['Sex_binary',target_body_size,'Age','Systolic_BP_mean_reading']

# Extract Pillai's trace values for each independent variable
pillai_values = [original_results.results[var]['stat']['Value'].loc['Pillai\'s trace'] for var in independent_vars]

# Create a DataFrame for easier plotting
pillai_df = pd.DataFrame({
    'Variable': independent_vars,
    'Pillai\'s Trace': pillai_values
})

# Use a clean style without grid and background
sns.set_style("white")

# Plot the Pillai's trace values using the Viridis color palette
plt.figure(figsize=(5, 3))
ax = sns.barplot(x=pillai_df['Pillai\'s Trace'], y=pillai_df['Variable'], color=color_pallete[3])

# Add annotations
for p in ax.patches:
    ax.annotate(f'{p.get_width():.2f}', (p.get_width() + 0.01, p.get_y() + p.get_height() / 2),
                ha='left', va='center', fontsize=14)
    ax.axvline(x=0.25, linestyle='--', color='gray', linewidth=1)
    ax.set_xlim(0, 0.6)

# Customize the plot
plt.xlabel('Pillai\'s Trace Value', fontsize=16)
plt.ylabel('')
plt.title('Model 3', fontsize=18)
plt.gca().invert_yaxis()  # To have the first variable at the top
plt.gca().xaxis.set_visible(False)  # Show the x-axis

# Adjust label names in the plot
label_dict = {'Sex_binary': 'Sex', 'Systolic_BP_mean_reading': 'Systolic BP', 'Age': 'Age',  target_body_size: target_body_size}  # Update to any variable names used
ax.set_yticklabels([label_dict.get(label, label) for label in pillai_df['Variable']])

# Remove all spines except the bottom (x-axis)
sns.despine(left=True, right=True, top=True, bottom=True)

# Display the plot
plt.show()

Performing MANCOVA of the shape coefficients against sex, age, systolic blood pressure (BP) and Weight: 

In [ ]:
target_body_size = 'Weight'

# Define the dependent variables
dependent_vars = [f'Mode_{mode}' for mode in range(1, 21)]

# Create formula string including all independent variables
formula = ' + '.join(dependent_vars) + ' ~ Sex_binary + '+ target_body_size +' + Age + Systolic_BP_mean_reading'  

# Perform MANOVA using from_formula
manova = MANOVA.from_formula(formula, data=full_df)

# Original MANOVA test results
original_results = manova.mv_test()

print(original_results)

# Extract p-values for the original data
original_p_values = {var: original_results.results[var]['stat']['Pr > F'].values[0] for var in original_results.results}

# List of independent variables in the model, with 'Sex_binary' renamed to 'Sex'
independent_vars = ['Sex_binary',target_body_size,'Age','Systolic_BP_mean_reading']

# Extract Pillai's trace values for each independent variable
pillai_values = [original_results.results[var]['stat']['Value'].loc['Pillai\'s trace'] for var in independent_vars]

# Create a DataFrame for easier plotting
pillai_df = pd.DataFrame({
    'Variable': independent_vars,
    'Pillai\'s Trace': pillai_values
})

# Use a clean style without grid and background
sns.set_style("white")

# Plot the Pillai's trace values using the Viridis color palette
plt.figure(figsize=(5, 3))
ax = sns.barplot(x=pillai_df['Pillai\'s Trace'], y=pillai_df['Variable'], color=color_pallete[4])

# Add annotations
for p in ax.patches:
    ax.annotate(f'{p.get_width():.2f}', (p.get_width() + 0.01, p.get_y() + p.get_height() / 2),
                ha='left', va='center', fontsize=14)
    ax.axvline(x=0.25, linestyle='--', color='gray', linewidth=1)
    ax.set_xlim(0, 0.6)

# Customize the plot
plt.xlabel('Pillai\'s Trace Value', fontsize=16)
plt.ylabel('')
plt.title('Model 4', fontsize=18)
plt.gca().invert_yaxis()  # To have the first variable at the top
plt.gca().xaxis.set_visible(False)  # Show the x-axis

# Adjust label names in the plot
label_dict = {'Sex_binary': 'Sex', 'Systolic_BP_mean_reading': 'Systolic BP', 'Age': 'Age',  target_body_size: target_body_size}  # Update to any variable names used
ax.set_yticklabels([label_dict.get(label, label) for label in pillai_df['Variable']])

# Remove all spines except the bottom (x-axis)
sns.despine(left=True, right=True, top=True, bottom=True)

# Display the plot
plt.show()

### 🛠️ Correcting the shape coefficients using multivariate linear regression. 

The shape coefficients are corrected by taking the residuals of the ordinary least square regression (OLS) model, trained against the covariates of interest. For more details see the relevant section of:  https://doi.org/10.1113/JP288667

In [ ]:
# List to store bmi_corrected shape scores
bmi_corrected_shape_scores = []
full_df['BMI_centered'] = full_df['BMI'] - full_df['BMI'].mean()
full_df['Systolic_BP_mean_reading_centered'] = full_df['Systolic_BP_mean_reading'] - full_df['Systolic_BP_mean_reading'].mean()
full_df['Age_centered'] = full_df['Age'] - full_df['Age'].mean()

# Define the independent variables you want to correct for
independent_vars = ['Age_centered', 'BMI_centered', 'Systolic_BP_mean_reading_centered']

# Loop through each shape mode and fit a single regressor (no sex-specific correction)
for mode in np.arange(1, 21):
    mode_column = f'Mode_{mode}'
    
    # Define the dependent variable (current shape mode)
    y = full_df[mode_column]
    
    # Define the independent variables (Age, BSA, MAP)
    X = full_df[independent_vars]
    
    # Add a constant to the model for intercept
    X = sm.add_constant(X)
    
    # Fit the linear regression model (correcting for Age, BSA, MAP)
    model = sm.OLS(y, X).fit()
    
    # Get the residuals, which are the bmi_corrected shape scores
    bmi_corrected_scores = model.resid
    
    # Store bmi_corrected shape scores along with the original index
    bmi_corrected_df = pd.DataFrame({
        'bmi_corrected_Shape_Score': bmi_corrected_scores,
        'Mode': mode,
        'Index': full_df.index
    })
    
    # Append to the list for concatenation later
    bmi_corrected_shape_scores.append(bmi_corrected_df)

# Concatenate all bmi_corrected shape scores for all modes
bmi_corrected_shape_scores_df = pd.concat(bmi_corrected_shape_scores)

# Reorder the bmi_corrected scores back to match the original data order
bmi_corrected_shape_scores_df = bmi_corrected_shape_scores_df.set_index('Index').sort_index()

# If you want to add the bmi_corrected shape scores back to your original dataframe
for mode in np.arange(1, 21):
    mode_column = f'Mode_{mode}'
    full_df[f'bmi_corrected_{mode_column}'] = bmi_corrected_shape_scores_df[bmi_corrected_shape_scores_df['Mode'] == mode]['bmi_corrected_Shape_Score'].values

# Check the first few rows to verify the bmi_corrected scores
full_df.head()

In [ ]:
# Perform Hotelling's T-squared test on the corrected shape scores:

# Filter data 

# Filter data for each sex
male_data = full_df[full_df['Sex'] == 'Male'][[f'bmi_corrected_Mode_{i}' for i in range(1, 21)]]
female_data = full_df[full_df['Sex'] == 'Female'][[f'bmi_corrected_Mode_{i}' for i in range(1, 21)]]

# Perform Hotelling's T-squared test
t_squared, f_value, p_value, pooled_variance = hotelling_t2(male_data, female_data)

# Display the results
print(f"Hotelling's T-squared statistic: {t_squared}")
print(f"F-statistic: {f_value}")
print(f"P-value: {p_value}")

In [ ]:
# List to store bsa_corrected shape scores
bsa_corrected_shape_scores = []

full_df['BSA_centered'] = full_df['BSA'] - full_df['BSA'].mean()

# Define the independent variables you want to correct for
independent_vars = ['Age_centered', 'BSA_centered', 'MAP']

# Loop through each shape mode and fit a single regressor (no sex-specific correction)
for mode in np.arange(1, 21):
    mode_column = f'Mode_{mode}'
    
    # Define the dependent variable (current shape mode)
    y = full_df[mode_column]
    
    # Define the independent variables (Age, BSA, MAP)
    X = full_df[independent_vars]
    
    # Add a constant to the model for intercept
    X = sm.add_constant(X)
    
    # Fit the linear regression model (correcting for Age, BSA, MAP)
    model = sm.OLS(y, X).fit()
    
    # Get the residuals, which are the bsa_corrected shape scores
    bsa_corrected_scores = model.resid
    
    # Store bsa_corrected shape scores along with the original index
    bsa_corrected_df = pd.DataFrame({
        'bsa_corrected_Shape_Score': bsa_corrected_scores,
        'Mode': mode,
        'Index': full_df.index
    })
    
    # Append to the list for concatenation later
    bsa_corrected_shape_scores.append(bsa_corrected_df)

# Concatenate all bsa_corrected shape scores for all modes
bsa_corrected_shape_scores_df = pd.concat(bsa_corrected_shape_scores)

# Reorder the bsa_corrected scores back to match the original data order
bsa_corrected_shape_scores_df = bsa_corrected_shape_scores_df.set_index('Index').sort_index()

# If you want to add the bsa_corrected shape scores back to your original dataframe
for mode in np.arange(1, 21):
    mode_column = f'Mode_{mode}'
    full_df[f'bsa_corrected_{mode_column}'] = bsa_corrected_shape_scores_df[bsa_corrected_shape_scores_df['Mode'] == mode]['bsa_corrected_Shape_Score'].values


# Perform Hotelling's T-squared test on the corrected shape scores:

# Filter data 

# Filter data for each sex
male_data = full_df[full_df['Sex'] == 'Male'][[f'bsa_corrected_Mode_{i}' for i in range(1, 21)]]
female_data = full_df[full_df['Sex'] == 'Female'][[f'bsa_corrected_Mode_{i}' for i in range(1, 21)]]

# Perform Hotelling's T-squared test
t_squared, f_value, p_value, pooled_variance = hotelling_t2(male_data, female_data)

# Display the results
print(f"Hotelling's T-squared statistic: {t_squared}")
print(f"F-statistic: {f_value}")
print(f"P-value: {p_value}")

In [ ]:
# List to store height_corrected shape scores
height_corrected_shape_scores = []

full_df['Height_centered'] = full_df['Height'] - full_df['Height'].mean()

# Define the independent variables you want to correct for
independent_vars = ['Age_centered', 'Height_centered', 'Systolic_BP_mean_reading_centered']

# Loop through each shape mode and fit a single regressor (no sex-specific correction)
for mode in np.arange(1, 21):
    mode_column = f'Mode_{mode}'
    
    # Define the dependent variable (current shape mode)
    y = full_df[mode_column]
    
    # Define the independent variables (Age, height, MAP)
    X = full_df[independent_vars]
    
    # Add a constant to the model for intercept
    X = sm.add_constant(X)
    
    # Fit the linear regression model (correcting for Age, height, MAP)
    model = sm.OLS(y, X).fit()
    
    # Get the residuals, which are the height_corrected shape scores
    height_corrected_scores = model.resid
    
    # Store height_corrected shape scores along with the original index
    height_corrected_df = pd.DataFrame({
        'height_corrected_Shape_Score': height_corrected_scores,
        'Mode': mode,
        'Index': full_df.index
    })
    
    # Append to the list for concatenation later
    height_corrected_shape_scores.append(height_corrected_df)

# Concatenate all height_corrected shape scores for all modes
height_corrected_shape_scores_df = pd.concat(height_corrected_shape_scores)

# Reorder the height_corrected scores back to match the original data order
height_corrected_shape_scores_df = height_corrected_shape_scores_df.set_index('Index').sort_index()

# If you want to add the height_corrected shape scores back to your original dataframe
for mode in np.arange(1, 21):
    mode_column = f'Mode_{mode}'
    full_df[f'height_corrected_{mode_column}'] = height_corrected_shape_scores_df[height_corrected_shape_scores_df['Mode'] == mode]['height_corrected_Shape_Score'].values

# Perform Hotelling's T-squared test on the corrected shape scores:

# Filter data 

# Filter data for each sex
male_data = full_df[full_df['Sex'] == 'Male'][[f'height_corrected_Mode_{i}' for i in range(1, 21)]]
female_data = full_df[full_df['Sex'] == 'Female'][[f'height_corrected_Mode_{i}' for i in range(1, 21)]]

# Perform Hotelling's T-squared test
t_squared, f_value, p_value, pooled_variance = hotelling_t2(male_data, female_data)

# Display the results
print(f"Hotelling's T-squared statistic: {t_squared}")
print(f"F-statistic: {f_value}")
print(f"P-value: {p_value}")

In [ ]:
# List to store weight_corrected shape scores
weight_corrected_shape_scores = []

full_df['Weight_centered'] = full_df['Weight'] - full_df['Weight'].mean()

# Define the independent variables you want to correct for
independent_vars = ['Age_centered', 'Weight_centered', 'Systolic_BP_mean_reading_centered']

# Loop through each shape mode and fit a single regressor (no sex-specific correction)
for mode in np.arange(1, 21):
    mode_column = f'Mode_{mode}'
    
    # Define the dependent variable (current shape mode)
    y = full_df[mode_column]
    
    # Define the independent variables (Age, weight, MAP)
    X = full_df[independent_vars]
    
    # Add a constant to the model for intercept
    X = sm.add_constant(X)
    
    # Fit the linear regression model (correcting for Age, weight, MAP)
    model = sm.OLS(y, X).fit()
    
    # Get the residuals, which are the weight_corrected shape scores
    weight_corrected_scores = model.resid
    
    # Store weight_corrected shape scores along with the original index
    weight_corrected_df = pd.DataFrame({
        'weight_corrected_Shape_Score': weight_corrected_scores,
        'Mode': mode,
        'Index': full_df.index
    })
    
    # Append to the list for concatenation later
    weight_corrected_shape_scores.append(weight_corrected_df)

# Concatenate all weight_corrected shape scores for all modes
weight_corrected_shape_scores_df = pd.concat(weight_corrected_shape_scores)

# Reorder the weight_corrected scores back to match the original data order
weight_corrected_shape_scores_df = weight_corrected_shape_scores_df.set_index('Index').sort_index()

# If you want to add the weight_corrected shape scores back to your original dataframe
for mode in np.arange(1, 21):
    mode_column = f'Mode_{mode}'
    full_df[f'weight_corrected_{mode_column}'] = weight_corrected_shape_scores_df[weight_corrected_shape_scores_df['Mode'] == mode]['weight_corrected_Shape_Score'].values


# Perform Hotelling's T-squared test on the corrected shape scores:

# Filter data 

# Filter data for each sex
male_data = full_df[full_df['Sex'] == 'Male'][[f'weight_corrected_Mode_{i}' for i in range(1, 21)]]
female_data = full_df[full_df['Sex'] == 'Female'][[f'weight_corrected_Mode_{i}' for i in range(1, 21)]]

# Perform Hotelling's T-squared test
t_squared, f_value, p_value, pooled_variance = hotelling_t2(male_data, female_data)

# Display the results
print(f"Hotelling's T-squared statistic: {t_squared}")
print(f"F-statistic: {f_value}")
print(f"P-value: {p_value}")

In [ ]:
full_df.to_excel(r"C:\Users\bmoscolo\OneDrive - UGent\PhD\WP1-SSM\Deformetrica\UKB_final_healthy_atlas_w_uoa\corrected_meshes/corrected_shape_scores.xlsx", index=False)

### 🎯 Logistic Regression Analysis:


The different correction schemes are evaluate by training logistic regression models on uncorrected and corrected coefficients. Receiving operator curves are analyzed to compare the correction schemes and evaluate the power of sex discrmination of cardiac morphology.

In [ ]:

# Define the independent variables (modes representing cardiac anatomy)
independent_vars = [f'Mode_{str(mode)}' for mode in np.arange(1, 21)]

# Add intercept term to independent variables
X = sm.add_constant(full_df[independent_vars])

# Encode the target variable 'Sex' as binary (0 for Female, 1 for Male)
full_df['Sex_binary'] = full_df['Sex'].map({'Female': 0, 'Male': 1})

# Fit logistic regression model
logit_model = sm.Logit(full_df['Sex_binary'], X)
results = logit_model.fit()

# Print summary including p-values
print(results.summary())

In [ ]:
# Define the corrected shape scores as independent variables
bmi_corrected_modes = [f'bmi_corrected_Mode_{mode}' for mode in np.arange(1, 21)]

# Extract the corrected shape scores from the dataframe
X_corrected = full_df[bmi_corrected_modes]

# Add intercept term to the corrected shape scores
X_corrected = sm.add_constant(X_corrected)

# Encode the target variable 'Sex' as binary (0 for Female, 1 for Male)
full_df['Sex_binary'] = full_df['Sex'].map({'Female': 0, 'Male': 1})

# Target variable 'Sex_binary' (assuming it's already binary coded: 0 for Female, 1 for Male)
y = full_df['Sex_binary']

# Fit the logistic regression model using the corrected shape scores
logit_model_bmi_corrected = sm.Logit(y, X_corrected)
result_bmi_corrected = logit_model_bmi_corrected.fit()

# Print summary including p-values
print(result_bmi_corrected.summary())
# Define the corrected shape scores as independent variables
bsa_corrected_modes = [f'bsa_corrected_Mode_{mode}' for mode in np.arange(1, 21)]

# Extract the corrected shape scores from the dataframe
X_corrected = full_df[bsa_corrected_modes]

# Add intercept term to the corrected shape scores
X_corrected = sm.add_constant(X_corrected)

# Encode the target variable 'Sex' as binary (0 for Female, 1 for Male)
full_df['Sex_binary'] = full_df['Sex'].map({'Female': 0, 'Male': 1})

# Target variable 'Sex_binary' (assuming it's already binary coded: 0 for Female, 1 for Male)
y = full_df['Sex_binary']

# Fit the logistic regression model using the corrected shape scores
logit_model_bsa_corrected = sm.Logit(y, X_corrected)
result_bsa_corrected = logit_model_bsa_corrected.fit()

# Print summary including p-values
print(result_bsa_corrected.summary())
# Define the corrected shape scores as independent variables
height_corrected_modes = [f'height_corrected_Mode_{mode}' for mode in np.arange(1, 21)]

# Extract the corrected shape scores from the dataframe
X_corrected = full_df[height_corrected_modes]

# Add intercept term to the corrected shape scores
X_corrected = sm.add_constant(X_corrected)

# Encode the target variable 'Sex' as binary (0 for Female, 1 for Male)
full_df['Sex_binary'] = full_df['Sex'].map({'Female': 0, 'Male': 1})

# Target variable 'Sex_binary' (assuming it's already binary coded: 0 for Female, 1 for Male)
y = full_df['Sex_binary']

# Fit the logistic regression model using the corrected shape scores
logit_model_height_corrected = sm.Logit(y, X_corrected)
result_height_corrected = logit_model_height_corrected.fit()

# Print summary including p-values
print(result_height_corrected.summary())
# Define the corrected shape scores as independent variables
weight_corrected_modes = [f'weight_corrected_Mode_{mode}' for mode in np.arange(1, 21)]

# Extract the corrected shape scores from the dataframe
X_corrected = full_df[weight_corrected_modes]

# Add intercept term to the corrected shape scores
X_corrected = sm.add_constant(X_corrected)

# Encode the target variable 'Sex' as binary (0 for Female, 1 for Male)
full_df['Sex_binary'] = full_df['Sex'].map({'Female': 0, 'Male': 1})

# Target variable 'Sex_binary' (assuming it's already binary coded: 0 for Female, 1 for Male)
y = full_df['Sex_binary']

# Fit the logistic regression model using the corrected shape scores
logit_model_weight_corrected = sm.Logit(y, X_corrected)
result_weight_corrected = logit_model_weight_corrected.fit()

# Print summary including p-values
print(result_weight_corrected.summary())

In [ ]:
bsa_corrected_modes = [f'bsa_corrected_Mode_{mode}' for mode in np.arange(1, 21)]
bmi_corrected_modes = [f'bmi_corrected_Mode_{mode}' for mode in np.arange(1, 21)]
height_corrected_modes = [f'height_corrected_Mode_{mode}' for mode in np.arange(1, 21)]
weight_corrected_modes = [f'weight_corrected_Mode_{mode}' for mode in np.arange(1, 21)]
dependent_vars = [f'Mode_{mode}' for mode in np.arange(1, 21)]

# Predict probabilities
X_non_adjusted = sm.add_constant(full_df[dependent_vars])
X_bsa_corrected = sm.add_constant(full_df[bsa_corrected_modes])
X_bmi_corrected = sm.add_constant(full_df[bmi_corrected_modes])
X_height_corrected = sm.add_constant(full_df[height_corrected_modes])
X_weight_corrected = sm.add_constant(full_df[weight_corrected_modes])

full_df['prob_non_adjusted'] = results.predict(X_non_adjusted)
full_df['prob_bsa_corrected'] = result_bmi_corrected.predict(X_bsa_corrected)
full_df['prob_bmi_corrected'] = result_bsa_corrected.predict(X_bmi_corrected)
full_df['prob_height_corrected'] = result_height_corrected.predict(X_height_corrected)
full_df['prob_weight_corrected'] = result_weight_corrected.predict(X_weight_corrected)

# Calculate ROC curves
fpr_non_adjusted, tpr_non_adjusted, _ = roc_curve(full_df['Sex_binary'], full_df['prob_non_adjusted'])
fpr_bsa_corrected, tpr_bsa_corrected, _ = roc_curve(full_df['Sex_binary'], full_df['prob_bsa_corrected'])
fpr_bmi_corrected, tpr_bmi_corrected, _ = roc_curve(full_df['Sex_binary'], full_df['prob_bmi_corrected'])
fpr_height_corrected, tpr_height_corrected, _ = roc_curve(full_df['Sex_binary'], full_df['prob_height_corrected'])
fpr_weight_corrected, tpr_weight_corrected, _ = roc_curve(full_df['Sex_binary'], full_df['prob_weight_corrected'])


# Calculate AUC
auc_non_adjusted = roc_auc_score(full_df['Sex_binary'], full_df['prob_non_adjusted'])
auc_bsa_corrected = roc_auc_score(full_df['Sex_binary'], full_df['prob_bsa_corrected'])
auc_bmi_corrected = roc_auc_score(full_df['Sex_binary'], full_df['prob_bmi_corrected'])
auc_height_corrected = roc_auc_score(full_df['Sex_binary'], full_df['prob_height_corrected'])
auc_weight_corrected = roc_auc_score(full_df['Sex_binary'], full_df['prob_weight_corrected'])   

# Set the viridis colormap
viridis = cm.get_cmap('viridis')

# Plot ROC curves using the viridis palette
plt.figure(figsize=(17, 17))
plt.plot(fpr_non_adjusted, tpr_non_adjusted, color=color_pallete[0], label=f'Non-Corrected Model (AUC = {auc_non_adjusted:.2f})', linewidth=6)
plt.plot(fpr_bmi_corrected, tpr_bmi_corrected, color=color_pallete[1], label=f'Age, BP and BMI Corrected Model (AUC = {auc_bmi_corrected:.2f})', linewidth=6)
plt.plot(fpr_bsa_corrected, tpr_bsa_corrected, color=color_pallete[3], label=f'Age, BP and BSA Corrected Model (AUC = {auc_bsa_corrected:.2f})', linewidth=6)
plt.plot(fpr_height_corrected, tpr_height_corrected, color=color_pallete[4], label=f'Age, BP and Height Corrected Model (AUC = {auc_height_corrected:.2f})', linewidth=6)
plt.plot(fpr_weight_corrected, tpr_weight_corrected, color=color_pallete[2], label=f'Age, BP and Weight Corrected Model (AUC = {auc_weight_corrected:.2f})', linewidth=6)
plt.plot([0, 1], [0, 1], color='grey', linestyle='--')  # Diagonal line for random guessing

# Add labels, title, and legend
plt.xlabel('(Proportion of) Females Misclassified as Males', fontsize=28)
plt.xticks(fontsize=20)
plt.ylabel('(Proportion of) Correctly Classified Males', fontsize=28)
plt.yticks(fontsize=20)
#plt.title('Male vs. Female Discriminative Power of Cardiac Morphology', fontsize = 20)
plt.legend(loc='lower right', fontsize=30)
plt.grid(False)

#plt.tight_layout()

# Remove borders (spines) from the plot
ax = plt.gca()  # Get current axis
for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_linewidth(0.5)
    spine.set_color('grey')


plt.show()